Před spuštěním ukázek je nejprve potřeba, spustit container s Redisem v Dockeru, dále pak následujicí kód. Ten implementuje do programu knihovnu Redis pro Python a spouští Redis na localhost na portu 6379

In [1]:
import redis                                                                                #přidání knihovny do programu

r = redis.Redis(host="localhost", port=6379, charset="utf-8", decode_responses=True)        #spuštění redis na localhostu na portu 6379

Mezi základní příkazy, umožňujicí standardní operace, které bychom od Redisu očekávali, lze jdenoznačně zařadit následujicí tři

Příkaz set("KEY", "VALUE")
pod zadaný KEY uloží data VALUE jako string, pokud KEY již VALUE obsahuje dojde dojde k jejímu přepsání

Příkaz get("KEY")
slouží k vypsání uložených dat (VALUE) pod zadaným klíčem (KEY), pokud KEY neexistuje vrátí příkaz hodnotu none

Příkaz delete("KEY")
smaže KEY, pokud KEY neexistuje neprovede nic

In [5]:
r.set("hello", "world")

value = r.get("hello")
print(value) # world

r.delete("hello")
print(r.get("hello")) # None

world
None


příkaz SET, má dále několik šikovných modifikací set("KEY", "VALUE", nx=True) umožňuje přiřadit hodnotu (VALUE) pouze pokud zadaný klíč (KEY) doposud neexistuje, protipólem tohoto příkazu je set("KEY", "VALUE", xx=True) který přepisuje hodnotu(VALUE) pouze již existujicímu klíči (KEY)

In [19]:
r.delete("ja") #ujistí se, že klíč není uložen, např při opětovném volání programu
r.set("ja", "clovek", xx=True) # pokusí se již existujicímu KEY přiřadit hodnotu, ten ovšem neexistuje, proto operace vrací none
print(r.get("ja")) # None

r.set("ja", "clovek", nx=True) # pokud KEY "ja" neexistuje vytvoří ho a přiřadí mu hodnotu "clovek"
print(r.get("ja")) # clovek

r.set("ja", "opice", xx=True) # pokusí se již existujicímu KEY přiřadit hodnotu, tentokrát se operace provede, jelikož klč již existuje
print(r.get("ja")) # opice

None
clovek
opice


další užitečnou modifikací příkazu SET je nastavení expirace KEY, expiraci můžeme přiřadit k již existujicímu KEY pomocí expire("KEY", "time in seconds"), nebo KEY expiraci přiřadit již při vytvoření pomocí set("KEY", "VALUE", ex="time in seconds"), k zobrazení zbývajicího času lze použít příkaz ttl("KEY")

In [36]:
r.set("programovani", "je super")
r.expire("programovani",10)
print(r.get("programovani"))


je super


In [37]:
r.ttl("programovani") #ukazuje dobu do expirace KEY programovani

8

In [38]:
print(r.get("programovani")) #v závislosti na době expirace vypíše VALUE nebo none

je super


variací příkazu GET je příkaz mget("KEY1","KEY2"), který dokáže vypsat více klíčů zároveň

In [42]:
r.set("telefon", "alcatel")
r.set("palivo", "jedine nafta")
print(r.mget("telefon", "palivo","nekdo")) #špatný KEY opět vypisuje hodnotu none

['alcatel', 'jedine nafta', None]


další možností při získávaní VALUE, je získání pouze jeho části k tomu slouží příkaz getrange("KEY", "od znaku","do znaku")

In [44]:
r.set("auto", "citroen se mi zrovna moc nelíbí")
print(r.getrange("auto", 0, 3))

citr


k vypsání existujicích klíčů lze požít příkaz scan("hodnota cusroru, pro vypisování hodnot") ten nevypisuje všechny klíče, ale pouze jejich část, v případě většího množství klíčů vrací funkce nejenom klíče, ale také hodnotu kursoru na od které lze pokračovat s vypisováním hodnot, pokud varcí hodnotu 0 byli již vypsány všechny klíče

In [49]:
r.set("tma", "svetlo")
r.scan(0)

(0, ['name', 'palivo', 'tma', 'ja', 'telefon', 'auto'])

VALUE může být i seznamem, podle toho zda chceme přidávat na konec či začátek využijeme příkaz lpush("KEY", "VALUE") pro začátek, či rpush("KEY", "VALUE") pro konec, příkaz lrange("KEY","od prvku","po prvek"))

In [64]:
r.delete("smer") #ujistí se, že klíč není uložen, např při opětovném volání programu
r.rpush("smer", "sever")
r.lpush("smer", "jih")#uloží hodnotu na první pozici
r.rpush("smer", "vychod")
r.rpush("smer", "zapad")
print(r.lrange("smer", 0, 2))#VALUE západ již nespadá do rozsahu příkazu, proto se nevypíše


['jih', 'sever', 'vychod']


dalšími zajimavými přkazy jsou příkazy incr("KEY"), zvyšování uložené hodnoty VALUE, decr("KEY"), snižování uložené hodnoty VALUE a incrby("KEY","VALUE"), zvyšování uložené hodnoty VALUE o počet

In [72]:
r.set("citac", 1)
r.incr("citac")
print(r.get("citac"))
r.incrby("citac",5)
print(r.get("citac"))
r.decr("citac")
print(r.get("citac"))

2
-3
-4


ukázky výše pokrývají a demonstrují základní příkazy jež lze v Rdisu využít, všechny jsou včetně dokukentace dostupné pod odkazem https://redis.io/commands